In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_excel("rolling_sale_data.xlsx")
print("Kích thước ban đầu:", df.shape)

Kích thước ban đầu: (79884, 21)


In [ ]:
# Lọc bỏ R* (condo units) ngoại trừ RR (CONDOMINIUM RENTALS)
mask_R  = df["BUILDING CLASS AT TIME OF SALE"].astype(str).str.startswith("R")
mask_RR = df["BUILDING CLASS AT TIME OF SALE"].astype(str).eq("RR")
raw_data = df[~mask_R].copy()
print("Sau khi bỏ R*:", raw_data.shape)
raw_data["BUILDING CLASS AT TIME OF SALE"].value_counts().head()

coop_classes = ["D4", "D0", "C6", "C8", "A8", "CC", "DC", "H7"]
mask_coop = raw_data["BUILDING CLASS AT TIME OF SALE"].astype(str).isin(coop_classes)
raw_data = raw_data[~mask_coop].copy()
print("Sau khi bỏ coop:", raw_data.shape)
raw_data["BUILDING CLASS AT TIME OF SALE"].value_counts().head()

raw_data = raw_data[raw_data["SALE PRICE"] > 0].copy()

Sau khi bỏ R*: (58816, 21)
Sau khi bỏ coop: (43072, 21)


In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24339 entries, 26658 to 79883
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   BOROUGH                         24339 non-null  int64         
 1   NEIGHBORHOOD                    24339 non-null  object        
 2   BUILDING CLASS CATEGORY         24339 non-null  object        
 3   TAX CLASS AT PRESENT            24339 non-null  object        
 4   BLOCK                           24339 non-null  int64         
 5   LOT                             24339 non-null  int64         
 6   EASEMENT                        0 non-null      float64       
 7   BUILDING CLASS AT PRESENT       24339 non-null  object        
 8   ADDRESS                         24339 non-null  object        
 9   APARTMENT NUMBER                0 non-null      object        
 10  ZIP CODE                        24334 non-null  float64       
 11  RES

In [5]:
def evaluate_regression_model(name, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae  = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2   = r2_score(y_test, y_pred)
    return {
        "model": name,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2
    }

## RAW

In [6]:
target_col = "SALE PRICE"

# === 0. Thêm biến thời gian từ SALE DATE ===
raw_data["SALE_YM"] = raw_data["SALE DATE"].dt.to_period("M").astype(str)   # dùng để split theo thời gian

raw_features = [
    "BOROUGH",
    "NEIGHBORHOOD",
    "BUILDING CLASS CATEGORY",
    "TAX CLASS AT PRESENT",
    "BLOCK",
    "LOT",
    "BUILDING CLASS AT PRESENT",
    "APARTMENT NUMBER",
    "ZIP CODE",
    "RESIDENTIAL UNITS",
    "COMMERCIAL UNITS",
    "TOTAL UNITS",
    "LAND SQUARE FEET",
    "GROSS SQUARE FEET",
    "YEAR BUILT",
    "TAX CLASS AT TIME OF SALE",
    "BUILDING CLASS AT TIME OF SALE",
]

# DataFrame cho kịch bản RAW
df_raw_scenario = raw_data[raw_features + [target_col, "SALE_YM"]].copy()

# === 1. Tách X, y (X chứa toàn bộ feature, chưa chia num/cat) ===
X_raw_full = df_raw_scenario.drop(columns=[target_col, "SALE_YM"])
y_raw      = df_raw_scenario[target_col]

# === 2. Xác định cột categorical / numeric ===
# Có thể chỉnh list này sau nếu muốn thêm/bớt cột category
cat_cols_raw = [
    "NEIGHBORHOOD",
    "BUILDING CLASS CATEGORY",
    "TAX CLASS AT PRESENT",
    "BUILDING CLASS AT PRESENT",
    "ZIP CODE",
    "TAX CLASS AT TIME OF SALE",
    "BUILDING CLASS AT TIME OF SALE",
]

# Giữ lại những cột cat thực sự có trong X_raw_full (tránh lỗi khi thiếu cột)
cat_cols_raw = [c for c in cat_cols_raw if c in X_raw_full.columns]

# Numeric = tất cả các cột còn lại
numeric_cols_raw = [c for c in X_raw_full.columns if c not in cat_cols_raw]

# Ép string cho categorical
for c in cat_cols_raw:
    X_raw_full[c] = X_raw_full[c].astype(str)

# === 3. Chia train/test theo 9 tháng đầu tiên (SALE_YM) ===
months_sorted = np.sort(df_raw_scenario["SALE_YM"].unique())
print("Các tháng (YEAR-MONTH) trong dữ liệu:", months_sorted)

train_months = months_sorted[:9]   # 9 tháng đầu của data
test_months  = months_sorted[9:]   # các tháng sau

train_mask = df_raw_scenario["SALE_YM"].isin(train_months)
test_mask  = df_raw_scenario["SALE_YM"].isin(test_months)

X_raw_train = X_raw_full[train_mask].copy()
X_raw_test  = X_raw_full[test_mask].copy()
y_raw_train = y_raw[train_mask].copy()
y_raw_test  = y_raw[test_mask].copy()

print("RAW train:", X_raw_train.shape, "RAW test:", X_raw_test.shape)

# === 4. PREPROCESSOR CHO RAW ===
numeric_transformer_raw = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_transformer_raw = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", drop="first")),
])

preprocess_raw = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_raw, numeric_cols_raw),
        ("cat", categorical_transformer_raw, cat_cols_raw),
    ],
    remainder="drop"   # cột nào không liệt kê ở trên sẽ bị drop
)   

Các tháng (YEAR-MONTH) trong dữ liệu: ['2024-11' '2024-12' '2025-01' '2025-02' '2025-03' '2025-04' '2025-05'
 '2025-06' '2025-07' '2025-08' '2025-09' '2025-10']
RAW train: (18883, 17) RAW test: (5456, 17)


## CLEAN

In [22]:
import numpy as np

clean_data = raw_data.copy()
clean_data["SALE DATE"] = pd.to_datetime(clean_data["SALE DATE"])

clean_data = clean_data[clean_data["SALE PRICE"] > 1000].copy()


# ==== 3. LỌC BASIC ====
# Sửa vài ZIP lỗi thủ công
clean_data.loc[28212, "ZIP CODE"] = 10305
clean_data.loc[28535, "ZIP CODE"] = 10308
clean_data.loc[30327, "ZIP CODE"] = 10309
clean_data.loc[78058, "ZIP CODE"] = 11356
clean_data.loc[78399, "ZIP CODE"] = 11415


clean_data["BOROUGH"] = clean_data["BOROUGH"].astype(str)
clean_data["TAX CLASS AT TIME OF SALE"] = clean_data["TAX CLASS AT TIME OF SALE"].astype(str)

# Tạo SALE_YEAR và SALE_MONTH từ SALE DATE
clean_data["SALE_YEAR"] = clean_data["SALE DATE"].dt.year
clean_data["SALE_MONTH"] = clean_data["SALE DATE"].dt.month

# ==== 4. ZIP CODE -> 3 SỐ ĐẦU, GIỮ NGUYÊN TÊN CỘT "ZIP CODE" ====
zip_digits = (
    clean_data["ZIP CODE"]
        .astype(str)
        .str.extract(r"(\d+)", expand=False)  # chuỗi số đầu tiên
)

mask_zip_valid = zip_digits.notna()
clean_data = clean_data[mask_zip_valid].copy()
zip_digits = zip_digits[mask_zip_valid]

clean_data["ZIP CODE"] = (
    zip_digits
        .str.zfill(5)   # "10001"
        .str[:3]        # "100"
)


# Bỏ các dòng SALE DATE bị NaT
clean_data = clean_data[clean_data["SALE DATE"].notna()].copy()

# ==== 5. TẠO SALE_YM VÀ CHIA TRAIN/TEST THEO THỜI GIAN ====
clean_data["SALE_YM"] = clean_data["SALE DATE"].dt.to_period("M").astype(str)
clean_data["BOROUGH"] = clean_data["BOROUGH"].astype(str)
months_sorted = np.sort(clean_data["SALE_YM"].unique())
print("Các tháng (YEAR-MONTH) trong CLEAN data:", months_sorted)

train_months = months_sorted[:9]   # 9 tháng đầu
test_months  = months_sorted[9:]   # các tháng sau

train_mask = clean_data["SALE_YM"].isin(train_months)
test_mask  = clean_data["SALE_YM"].isin(test_months)

train_clean = clean_data[train_mask].copy()
test_clean  = clean_data[test_mask].copy()

print("CLEAN train:", train_clean.shape, "CLEAN test:", test_clean.shape)

# ==== 5b. ROLLING SALE PRICE 7 / 14 DAY THEO ZIP CODE, LÀM RIÊNG CHO TRAIN & TEST ====

def add_rolling_features(df):
    df = df.sort_values(["ZIP CODE", "SALE DATE"])

    def _rolling(g):
        g = g.sort_values("SALE DATE")

        s = g.set_index("SALE DATE")["SALE PRICE"]

        # Rolling 7 ngày và 14 ngày, chỉ dùng PAST (shift(1))
        roll7 = s.rolling("7D").mean().shift(1)
        roll14 = s.rolling("14D").mean().shift(1)

        g["ROLL_MEAN_SP_7D"] = roll7.values
        g["ROLL_MEAN_SP_14D"] = roll14.values
        return g

    return (
        df.groupby("ZIP CODE", group_keys=False)
          .apply(_rolling)
    )

train_clean = add_rolling_features(train_clean)
test_clean  = add_rolling_features(test_clean)

# ==== 6. FILL YEAR BUILT THEO MOST FREQUENT CỦA ZIP CODE (CHỈ DÙNG TRAIN) ====
zip_year_mode = (
    train_clean
    .groupby("ZIP CODE")["YEAR BUILT"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.median())
)

global_year_median = train_clean["YEAR BUILT"].median()

mask_train_na = train_clean["YEAR BUILT"].isna()
train_clean.loc[mask_train_na, "YEAR BUILT"] = (
    train_clean.loc[mask_train_na, "ZIP CODE"].map(zip_year_mode)
)
train_clean["YEAR BUILT"] = train_clean["YEAR BUILT"].fillna(global_year_median)

mask_test_na = test_clean["YEAR BUILT"].isna()
test_clean.loc[mask_test_na, "YEAR BUILT"] = (
    test_clean.loc[mask_test_na, "ZIP CODE"].map(zip_year_mode).fillna(global_year_median)
)

# ==== 6b. TÍNH BUILDING AGE SAU KHI ĐÃ FILL YEAR BUILT ====
train_clean["BUILDING AGE"] = train_clean["SALE_YEAR"] - train_clean["YEAR BUILT"]
test_clean["BUILDING AGE"]  = test_clean["SALE_YEAR"] - test_clean["YEAR BUILT"]

# ==== 6c. LOG-TRANSFORM SALE PRICE + 2 BIẾN FEET ====
# target log
train_clean["LOG_SALE_PRICE"] = np.log1p(train_clean["SALE PRICE"])
test_clean["LOG_SALE_PRICE"]  = np.log1p(test_clean["SALE PRICE"])

# feature log (feet)
train_clean["LOG_LAND_SQFT"]  = np.log1p(train_clean["LAND SQUARE FEET"])
test_clean["LOG_LAND_SQFT"]   = np.log1p(test_clean["LAND SQUARE FEET"])

train_clean["LOG_GROSS_SQFT"] = np.log1p(train_clean["GROSS SQUARE FEET"])
test_clean["LOG_GROSS_SQFT"]  = np.log1p(test_clean["GROSS SQUARE FEET"])

# ==== 7. GIỮ LẠI CLEAN FEATURES ====
clean_features = [
    "BOROUGH",
    "TAX CLASS AT TIME OF SALE",
    "BUILDING CLASS CATEGORY",
    "RESIDENTIAL UNITS",
    "COMMERCIAL UNITS",
    "LOG_LAND_SQFT",
    "LOG_GROSS_SQFT",
    "YEAR BUILT",
    "BUILDING AGE",
    "ZIP CODE",              # 3 số đầu, dùng như categorical
    "SALE_MONTH",
    "ROLL_MEAN_SP_7D",
    "ROLL_MEAN_SP_14D",
    "SALE DATE",
    "SALE PRICE",
    "LOG_SALE_PRICE",
]

train_clean = train_clean[clean_features].copy()
test_clean  = test_clean[clean_features].copy()

train_clean.info()
test_clean.info()


Các tháng (YEAR-MONTH) trong CLEAN data: ['2024-11' '2024-12' '2025-01' '2025-02' '2025-03' '2025-04' '2025-05'
 '2025-06' '2025-07' '2025-08' '2025-09' '2025-10']
CLEAN train: (17824, 24) CLEAN test: (5238, 24)
<class 'pandas.core.frame.DataFrame'>
Index: 17824 entries, 73870 to 43198
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   BOROUGH                    17824 non-null  object        
 1   TAX CLASS AT TIME OF SALE  17824 non-null  object        
 2   BUILDING CLASS CATEGORY    17824 non-null  object        
 3   RESIDENTIAL UNITS          17824 non-null  float64       
 4   COMMERCIAL UNITS           17824 non-null  float64       
 5   LOG_LAND_SQFT              17824 non-null  float64       
 6   LOG_GROSS_SQFT             17824 non-null  float64       
 7   YEAR BUILT                 17824 non-null  float64       
 8   BUILDING AGE               17824 non-null  fl

In [44]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold


target_col = "LOG_SALE_PRICE"   # dùng log sale price làm target

feature_cols_clean = [
    "TAX CLASS AT TIME OF SALE",
    "BUILDING CLASS CATEGORY",
    "RESIDENTIAL UNITS",
    "COMMERCIAL UNITS",
    "LOG_LAND_SQFT",     # log feet mới
    "LOG_GROSS_SQFT",    # log feet mới
    "BUILDING AGE",
    "ZIP CODE",          # 3 số đầu
    "SALE_MONTH",
    "ROLL_MEAN_SP_7D",
    "ROLL_MEAN_SP_14D",
    ]

X_clean_train = train_clean[feature_cols_clean].copy()
y_clean_train = train_clean[target_col].copy()

X_clean_test  = test_clean[feature_cols_clean].copy()
y_clean_test  = test_clean[target_col].copy()

# =========================
# 2. Cột categorical / numeric
# =========================
cat_cols_clean = [
    "TAX CLASS AT TIME OF SALE",
    "BUILDING CLASS CATEGORY",
    "ZIP CODE",

]

numeric_cols_clean = [c for c in feature_cols_clean if c not in cat_cols_clean]
print("Numeric CLEAN:", numeric_cols_clean)
print("Categorical CLEAN:", cat_cols_clean)

# =========================
# 3. PREPROCESSOR cho CLEAN
# =========================
numeric_transformer_clean = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

categorical_transformer_clean = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

col_transformer_clean = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_clean, numeric_cols_clean),
        ("cat", categorical_transformer_clean, cat_cols_clean),
    ],
    remainder="drop",
)

# Bọc ColumnTransformer + VarianceThreshold vào 1 pipeline
preprocess_clean = Pipeline(steps=[
    ("columns", col_transformer_clean),
    # threshold=0.0: loại các cột có variance = 0 (toàn 0 hoặc toàn 1)
    # nếu muốn loại cả cột cực kỳ hiếm, có thể tăng lên 0.01, 0.001,...
    ("var_thresh", VarianceThreshold(threshold=0.001)),
])



Numeric CLEAN: ['RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'LOG_LAND_SQFT', 'LOG_GROSS_SQFT', 'BUILDING AGE', 'SALE_MONTH', 'ROLL_MEAN_SP_7D', 'ROLL_MEAN_SP_14D']
Categorical CLEAN: ['TAX CLASS AT TIME OF SALE', 'BUILDING CLASS CATEGORY', 'ZIP CODE']


In [26]:
# Base models (chắc chắn có trong sklearn)
models = {

    # Tree & Ensemble
    "DecisionTree": DecisionTreeRegressor(max_depth=10, random_state=42),
}


In [10]:
def evaluate_regression_model_log(name, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred = np.expm1(y_pred)
    y_test = np.expm1(y_test)
    mae  = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2   = r2_score(y_test, y_pred)
    return {
        "model": name,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2
    }

In [45]:
results = []

# 6.1. RAW (y giữ nguyên scale gốc)
for name, base_model in models.items():
    pipe_raw = Pipeline(steps=[
        ("preprocess", preprocess_raw),
        ("model", base_model),
    ])
    
    res_raw = evaluate_regression_model(
        name=f"{name}_RAW",
        model=pipe_raw,
        X_train=X_raw_train,
        X_test=X_raw_test,
        y_train=y_raw_train,
        y_test=y_raw_test,
    )
    results.append(res_raw)

# 6.2. CLEAN 
for name, base_model in models.items():
    pipe_clean = Pipeline(steps=[
        ("preprocess", preprocess_clean),
        ("model", base_model),
    ])
    
    res_clean = evaluate_regression_model_log(
        name=f"{name}_CLEAN",
        model=pipe_clean,
        X_train=X_clean_train,
        X_test=X_clean_test,
        y_train=y_clean_train,
        y_test=y_clean_test,
    )
    results.append(res_clean)

# Tổng hợp kết quả
results_df = pd.DataFrame(results)

# Tách model base và scenario (RAW/CLEAN) cho dễ đọc
results_df["BaseModel"] = results_df["model"].str.replace("_(RAW|CLEAN)$", "", regex=True)
results_df["Scenario"]  = results_df["model"].str.extract("_(RAW|CLEAN)$")[0]

# Sắp xếp cho dễ so sánh: theo BaseModel rồi Scenario
results_df = results_df.sort_values(["BaseModel", "Scenario"]).reset_index(drop=True)
results_df


,model,MAE,RMSE,R2,BaseModel,Scenario
0,DecisionTree_CLEAN,2.071482e+06,2.660531e+07,0.233057,DecisionTree,CLEAN
1,DecisionTree_RAW,2.237606e+06,2.902625e+07,0.049542,DecisionTree,RAW
